In [1]:
import geopandas as gpd
import pandas as pd
import requests
import json

In [2]:
# Walkability by census block group
# src: https://www.epa.gov/smartgrowth/smart-location-mapping#walkability
gdf = gpd.read_file("WalkabilityIndex/Natl_WI.gdb", driver='FileGDB', layer='NationalWalkabilityIndex')

# Limit to just Austin metro area
gdf = gdf[gdf["CBSA_Name"] == "Austin-Round Rock-Georgetown, TX"]

# score walkability
# Higher walkability -> higher score
gdf["walkability_rank"] = gdf["NatWalkInd"].rank()

In [3]:
# get existing metrobike kiosk data
# src: https://data.austintexas.gov/d/qd73-bsdg

kiosks_existing = pd.read_csv("https://data.austintexas.gov/resource/qd73-bsdg.csv?$select=kiosk_id,kiosk_name,location.latitude,location.longitude,kiosk_status")
kiosks_existing = gpd.GeoDataFrame(
    kiosks_existing, geometry=gpd.points_from_xy(kiosks_existing["location_longitude"], kiosks_existing["location_latitude"]))

# Filtering out non-active kiosks
kiosks_existing = kiosks_existing[kiosks_existing["kiosk_status"] == "active"]

In [4]:
# dealing w/ coordinate reference systems...
kiosks_existing = kiosks_existing.set_crs(4326)
gdf = gdf.to_crs(4326)

In [5]:
# calculating the number of kiosks in each census block group

# Gets polygon ID by kiosk
points_in_polygons = gpd.sjoin(kiosks_existing, gdf, how="left", op="within")
# Grouping by census block group
point_counts = points_in_polygons.groupby('GEOID10').size()

# rejoining data back to our original dataframe
df = pd.DataFrame(point_counts)
df = df.reset_index()

gdf = gdf.merge(df, on="GEOID10", how="left")
gdf = gdf.rename(columns={0:"count_ex_kiosks"})
gdf["count_ex_kiosks"] = gdf["count_ex_kiosks"].fillna(0)

/Users/charliehenry/anaconda3/envs/atd/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [6]:
## Transit Scoring
# D4A is the distance in meters to the nearest transit stop

# We want these unranked zones to be assumed far from transit 
gdf["D4A"] = gdf["D4A"].replace(-99999.00, 99999.00)
# closer distance == higher rank
gdf["transit_ranking"] = gdf["D4A"].rank(ascending=False)

In [7]:
## Previous Micromobility Trips (tract level)
# src: https://data.austintexas.gov/d/7d8e-dm7r

# trip starts
trips = pd.read_csv("https://data.austintexas.gov/resource/7d8e-dm7r.csv?$query=SELECT%20%60census_geoid_start%60%2C%20count(%60trip_id%60)%20AS%20%60count_trip_id%60%0AGROUP%20BY%20%60census_geoid_start%60%0AHAVING%0A%20%20caseless_not_one_of(%0A%20%20%20%20%60census_geoid_start%60%2C%0A%20%20%20%20%22%22%2C%0A%20%20%20%20%220%22%2C%0A%20%20%20%20%22None%22%2C%0A%20%20%20%20%22OUT_OF_BOUNDS%22%0A%20%20)")
# trip ends
tripe = pd.read_csv("https://data.austintexas.gov/resource/7d8e-dm7r.csv?$query=SELECT%20%60census_geoid_end%60%2C%20count(%60trip_id%60)%20AS%20%60count_trip_id%60%0AGROUP%20BY%20%60census_geoid_end%60%0AHAVING%0A%20%20caseless_not_one_of(%0A%20%20%20%20%60census_geoid_end%60%2C%0A%20%20%20%20%22%22%2C%0A%20%20%20%20%220%22%2C%0A%20%20%20%20%22None%22%2C%0A%20%20%20%20%22OUT_OF_BOUNDS%22%0A%20%20)") 

In [8]:
# Creating column we will join
gdf["census_tract_geoid"] = gdf["STATEFP"]+ gdf["COUNTYFP"] + gdf["TRACTCE"]
gdf["census_tract_geoid"] = gdf["census_tract_geoid"].astype(int)

# joining in trip counts
gdf = gdf.merge(trips, left_on="census_tract_geoid", right_on="census_geoid_start", how="left")
gdf = gdf.merge(tripe, left_on="census_tract_geoid", right_on="census_geoid_end", how="left")
gdf["count_micromobility_trips"] = gdf["count_trip_id_x"].fillna(0) + gdf["count_trip_id_y"].fillna(0)

# more trips -> higher score
gdf["micromobility_score"] = gdf["count_micromobility_trips"].rank()


In [9]:
## get equity analysis zones (tract level)
# src: https://austin.maps.arcgis.com/home/item.html?id=0a095a37ea8a4eb8b835a888f00ef53f&sublayer=0#overview
res = requests.get("https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/Equity_Analysis_Zones_2021/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&returnEnvelope=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token=")
eaz = json.loads(res.text)
eaz = pd.DataFrame([f["attributes"] for f in eaz["features"]])

# cleaning up data before merging
eaz = eaz.rename(columns={"GEOID": "census_tract_geoid", "indxd_v": "vulnerability_index"})
eaz = eaz[["census_tract_geoid","vulnerability_index"]]
eaz["census_tract_geoid"] = eaz["census_tract_geoid"].astype(int)

# merge in EAZs
gdf = gdf.merge(eaz, on="census_tract_geoid", how="left")
gdf["vulnerability_index"] = gdf["vulnerability_index"].fillna(0)
gdf["vulnerability_rank"] = gdf["vulnerability_index"].rank()
# higher vulnerability -> higher score

In [10]:
# before min-max normalization:
print(gdf["transit_ranking"].min(), gdf["transit_ranking"].max())
print(gdf["walkability_rank"].min(), gdf["walkability_rank"].max())
print(gdf["micromobility_score"].min(), gdf["micromobility_score"].max())
print(gdf["vulnerability_rank"].min(), gdf["vulnerability_rank"].max())

275.0 965.0
1.5 967.0
44.0 966.5
6.5 966.0


In [11]:
# min-max normalize all rankings
def normalize_ranking(df, col):
    max = df[col].max()
    min = df[col].min()
    df[f"{col}_norm"] = df[col] - min
    df[f"{col}_norm"] = df[f"{col}_norm"]/(max-min)
    df[f"{col}_norm"] = df[f"{col}_norm"] * 100

In [12]:
normalize_ranking(gdf, "transit_ranking")
normalize_ranking(gdf, "walkability_rank")
normalize_ranking(gdf, "micromobility_score")
normalize_ranking(gdf, "vulnerability_rank")

In [13]:
# after min-max normalization
print(gdf["transit_ranking_norm"].min(), gdf["transit_ranking_norm"].max())
print(gdf["walkability_rank_norm"].min(), gdf["walkability_rank_norm"].max())
print(gdf["micromobility_score_norm"].min(), gdf["micromobility_score_norm"].max())
print(gdf["vulnerability_rank_norm"].min(), gdf["vulnerability_rank_norm"].max())

0.0 100.0
0.0 100.0
0.0 100.0
0.0 100.0


In [14]:
# playing with different weights
transit_weight = 0.35
walkability_weight = 0.20
micromobility_weight = 0.35
equity_weight = 0.10

In [15]:
# get metrobike score
gdf["metrobike_score"] = transit_weight*gdf["transit_ranking_norm"] + walkability_weight*gdf["walkability_rank_norm"] + micromobility_weight*gdf["micromobility_score_norm"] + equity_weight*gdf["vulnerability_rank_norm"]

In [16]:
# plotting the top 20
sample = gdf[gdf["count_ex_kiosks"]==0].sort_values("metrobike_score", ascending=False).head(20)
m = sample.explore(name="Polygons")
m = kiosks_existing.explore(m=m, color="red", name="Points")
m

In [17]:
# exporting results
gdf.to_file("metrobike_kiosk_analysis.geojson")